In [1]:
import pandas as pd
df=pd.read_csv(r"E:\MY projects\Amazon project\full_amazon_project\Amazon-web-app\amz_ca_total_products_data_processed.csv")
df

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,4.4,2876,47.69,0.00,Industrial Scientific,False,0
1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,3.8,55,10.99,0.00,Industrial Scientific,False,100
2,B087P7538J,YaeCCC 19 Pcs Hole Saw Kit 3/4''(19mm)- 6''(15...,https://m.media-amazon.com/images/I/71P+qLFDFc...,https://www.amazon.ca/dp/B087P7538J,4.0,126,25.99,27.99,Industrial Scientific,False,50
3,B0822FF7YQ,LLPT Butyl Putty Tape White 1 Inch x 33Ft Tigh...,https://m.media-amazon.com/images/I/61wfWLZRn-...,https://www.amazon.ca/dp/B0822FF7YQ,4.5,1936,21.99,30.99,Industrial Scientific,False,100
4,B08NTR8NN4,"Lightbeam 16"" Long Stem Deep Fry Thermometer w...",https://m.media-amazon.com/images/I/61qJG-y1He...,https://www.amazon.ca/dp/B08NTR8NN4,4.2,46,18.99,0.00,Industrial Scientific,False,100
...,...,...,...,...,...,...,...,...,...,...,...
2165921,B0B9Y4PJZ2,"Climbing Rope Net for Kids, Net Climbing Cargo...",https://m.media-amazon.com/images/I/81c39KcQWI...,https://www.amazon.ca/dp/B0B9Y4PJZ2,0.0,0,52.99,0.00,Play Sets Playground Equipment,False,0
2165922,B0B9XBN868,YIJU Gymnastic Rings Workout Set with Adjustab...,https://m.media-amazon.com/images/I/61p9ROOs62...,https://www.amazon.ca/dp/B0B9XBN868,0.0,0,15.99,0.00,Play Sets Playground Equipment,False,0
2165923,B0B9T78NVK,Fancyes 2X Heavy Duty Swing Hangers Hardware S...,https://m.media-amazon.com/images/I/51VdGcyjqO...,https://www.amazon.ca/dp/B0B9T78NVK,0.0,0,15.29,0.00,Play Sets Playground Equipment,False,0
2165924,B0B9T41CCT,SM SunniMix 2Pcs Stainless Steel Swing Hangers...,https://m.media-amazon.com/images/I/51o23h9tR9...,https://www.amazon.ca/dp/B0B9T41CCT,0.0,0,21.09,0.00,Play Sets Playground Equipment,False,0


In [12]:
import pandas as pd
import numpy as np
import flask
import sentence_transformers
import sklearn
import sys

print("Python Version:", sys.version)
print("Pandas Version:", pd.__version__)
print("NumPy Version:", np.__version__)
print("Flask Version:", flask.__version__)
print("SentenceTransformers Version:", sentence_transformers.__version__)
print("scikit-learn Version:", sklearn.__version__)


Python Version: 3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]
Pandas Version: 2.2.3
NumPy Version: 1.26.4
Flask Version: 3.1.0
SentenceTransformers Version: 4.0.2
scikit-learn Version: 1.6.1


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_9256\2824441776.py:11: DeprecationWarning: The '__version__' attribute is deprecated and will be removed in Flask 3.1. Use feature detection or 'importlib.metadata.version("flask")' instead.
  print("Flask Version:", flask.__version__)


In [2]:
df.columns

Index(['asin', 'title', 'imgUrl', 'productURL', 'stars', 'reviews', 'price',
       'listPrice', 'categoryName', 'isBestSeller', 'boughtInLastMonth'],
      dtype='object')

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 2. Drop duplicates or NaNs if any
df = df.dropna(subset=['title']).drop_duplicates(subset=['title'])

# 3. Load the sentence transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# 4. Encode all product names into embeddings
name_embeddings = model.encode(df['title'].tolist(), show_progress_bar=True)

# 5. Function to get top-k similar products based on input name
def get_recommendations(input_name, top_k=10):
    # Encode the input name
    input_embedding = model.encode([input_name])
    
    # Compute cosine similarity with all embeddings
    similarities = cosine_similarity(input_embedding, name_embeddings)[0]
    
    # Get top k indices (excluding the input itself if it's in the list)
    top_indices = np.argsort(similarities)[::-1][1:top_k+1]
    
    # Return the most similar product names with their scores
    recommendations = df.iloc[top_indices][['title','imgUrl', 'productURL']].copy()
    recommendations['similarity'] = similarities[top_indices]
    
    return recommendations.reset_index(drop=True)

# 6. Example usage:
query = "Wireless Bluetooth Headphones Earphones"
recommendations = get_recommendations(query, top_k=10)

# 7. View results
print(recommendations)

Batches:   0%|          | 0/275 [00:00<?, ?it/s]

                                                name  \
0   Techno Simba Buds - Bluetooth Wireless Earphones   
1  Noise Sense Bluetooth Wireless in Ear Earphone...   
2  Mobomart Pro 5 Bluetooth Truly Wireless Ear Ea...   
3  Wireless Bluetooth Headphones Earphones for vi...   
4  M19 Wireless Earbuds TWS 5.1 Large Screen Dual...   
5  truke Buds BTG3 Bluetooth Truly Wireless in Ea...   
6  Sony WH-CH510 Wireless Bluetooth On Ear Headph...   
7  Samsung Galaxy Buds2 Pro, Bluetooth Truly Wire...   
8  Noise Beads Bluetooth Truly Wireless in Ear Ea...   
9  Noise Beads Bluetooth Truly Wireless in Ear Ea...   

                                               image  \
0  https://m.media-amazon.com/images/I/51sGq1BiVU...   
1  https://m.media-amazon.com/images/I/515Ms0UVcO...   
2  https://m.media-amazon.com/images/W/IMAGERENDE...   
3  https://m.media-amazon.com/images/I/51dQRgJ+qx...   
4  https://m.media-amazon.com/images/W/IMAGERENDE...   
5  https://m.media-amazon.com/images/W/IMAGEREN

In [3]:
# Step 1: Imports
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display
import ipywidgets as widgets

# Step 2: Load and clean dataset
df = pd.read_csv('amz_ca_total_products_data_processed.csv')
df = df.dropna(subset=['title', 'categoryName']).drop_duplicates(subset=['title'])

# Step 3: Extract unique categories
categories = sorted(df['categoryName'].unique().tolist())

# Step 4: Load the sentence transformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Step 5: Define recommendation function
def get_recommendations(input_name, category, top_k=10):
    filtered_df = df[df['categoryName'] == category].reset_index(drop=True)
    
    if filtered_df.empty:
        return pd.DataFrame()

    title_list = filtered_df['title'].tolist()
    name_embeddings = model.encode(title_list, show_progress_bar=False)
    input_embedding = model.encode([input_name])

    similarities = cosine_similarity(input_embedding, name_embeddings)[0]
    top_indices = np.argsort(similarities)[::-1][:top_k]

    recommendations = filtered_df.iloc[top_indices][['title', 'imgUrl', 'productURL']].copy()
    recommendations['similarity'] = similarities[top_indices]
    return recommendations.reset_index(drop=True)

# Step 6: Interactive widgets
category_dropdown = widgets.Dropdown(
    options=categories,
    description='Category:',
    layout=widgets.Layout(width='50%')
)

query_text = widgets.Text(
    description='Query:',
    placeholder='Enter product name',
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

def on_search_click(b):
    output.clear_output()
    with output:
        category = category_dropdown.value
        query = query_text.value.strip()
        if not query:
            print("Please enter a product name.")
            return
        results = get_recommendations(query, category)
        if results.empty:
            print("No results found in this category.")
        else:
            display(results)

search_button = widgets.Button(description='Search', button_style='success')
search_button.on_click(on_search_click)

# Display widgets
display(category_dropdown, query_text, search_button, output)


Dropdown(description='Category:', layout=Layout(width='50%'), options=('3D Printing  Scanning', 'Abrasive  Fin…

Text(value='', description='Query:', layout=Layout(width='50%'), placeholder='Enter product name')

Button(button_style='success', description='Search', style=ButtonStyle())

Output()

In [9]:
pd.set_option('display.max_colwidth', None)  # To show full strings in notebook


In [11]:
print(df['name'].head(5))  # See if product names are complete


0                                                                             Redmi 10 Power (Power Black, 8GB RAM, 128GB Storage)
1                                                                    OnePlus Nord CE 2 Lite 5G (Blue Tide, 6GB RAM, 128GB Storage)
2    OnePlus Bullets Z2 Bluetooth Wireless in Ear Earphones with Mic, Bombastic Bass - 12.4 Mm Drivers, 10 Mins Charge - 20 Hrs...
3    Samsung Galaxy M33 5G (Mystique Green, 6GB, 128GB Storage) | 6000mAh Battery | Upto 12GB RAM with RAM Plus | Travel Adapte...
4                                                                   OnePlus Nord CE 2 Lite 5G (Black Dusk, 6GB RAM, 128GB Storage)
Name: name, dtype: object


In [7]:
import numpy as np

# Compute embeddings once
name_embeddings = model.encode(df['name'].tolist(), show_progress_bar=True)

# Save them
np.save("name_embeddings.npy", name_embeddings)
df.to_csv("cleaned_product_data.csv", index=False)

Batches:   0%|          | 0/275 [00:00<?, ?it/s]

In [1]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# Load and clean dataset
df = pd.read_csv('amz_ca_total_products_data_processed.csv')
df = df.dropna(subset=['title', 'categoryName']).drop_duplicates(subset=['title'])
categories = sorted(df['categoryName'].unique().tolist())

# Load SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Normalize for FAISS cosine similarity
def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

# Build FAISS HNSW index per category
category_indices = {}
category_id_maps = {}

for category in categories:
    cat_df = df[df['categoryName'] == category].reset_index(drop=True)
    if not cat_df.empty:
        titles = cat_df['title'].tolist()
        embeddings = model.encode(titles, show_progress_bar=False)
        embeddings = normalize(np.array(embeddings).astype('float32'))
        
        dim = embeddings.shape[1]
        index = faiss.IndexHNSWFlat(dim, 32)
        index.hnsw.efSearch = 50
        index.hnsw.efConstruction = 40
        index.add(embeddings)
        
        category_indices[category] = index
        category_id_maps[category] = cat_df


KeyboardInterrupt: 

In [ ]:
def get_recommendations(input_name, category, top_k=10):
    if category not in category_indices:
        return pd.DataFrame()
    
    input_embedding = model.encode([input_name])
    input_embedding = normalize(np.array(input_embedding).astype('float32'))

    index = category_indices[category]
    distances, indices = index.search(input_embedding, top_k)
    
    df_cat = category_id_maps[category]
    results = df_cat.iloc[indices[0]][['title', 'imgUrl', 'productURL']].copy()
    results['similarity'] = 1 - distances[0]
    return results.reset_index(drop=True)


In [ ]:
input_query = "Phone Case"
category = "Electronics"

recommendations = get_recommendations(input_query, category)
recommendations


In [4]:
import os
import pickle
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

# --- Helper functions for saving/loading ---
import re

def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "_", name)

def save_faiss_index(index, category):
    safe_cat = sanitize_filename(category)
    os.makedirs("faiss_indexes", exist_ok=True)
    faiss.write_index(index, f"faiss_indexes/{safe_cat}.index")

def load_faiss_index(category):
    safe_cat = sanitize_filename(category)
    return faiss.read_index(f"faiss_indexes/{safe_cat}.index")

def save_category_data(df_cat, category):
    safe_cat = sanitize_filename(category)
    os.makedirs("faiss_indexes", exist_ok=True)
    with open(f"faiss_indexes/{safe_cat}_data.pkl", "wb") as f:
        pickle.dump(df_cat, f)

def load_category_data(category):
    safe_cat = sanitize_filename(category)
    with open(f"faiss_indexes/{safe_cat}_data.pkl", "rb") as f:
        return pickle.load(f)


def normalize(vectors):
    return vectors / np.linalg.norm(vectors, axis=1, keepdims=True)

# --- Load and clean dataset ---
df = pd.read_csv('amz_ca_total_products_data_processed.csv')
df = df.dropna(subset=['title', 'categoryName']).drop_duplicates(subset=['title'])
categories = sorted(df['categoryName'].unique().tolist())

# --- Load multilingual sentence transformer model ---
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# --- Load or build FAISS indexes and category data ---
category_indices = {}
category_id_maps = {}

for category in categories:
    index_path = f"faiss_indexes/{category}.index"
    data_path = f"faiss_indexes/{category}_data.pkl"
    
    if os.path.exists(index_path) and os.path.exists(data_path):
        print(f"Loading index and data for category: {category}")
        index = load_faiss_index(category)
        cat_df = load_category_data(category)
    else:
        print(f"Building index and data for category: {category}")
        cat_df = df[df['categoryName'] == category].reset_index(drop=True)
        if not cat_df.empty:
            titles = cat_df['title'].tolist()
            embeddings = model.encode(titles, show_progress_bar=False)
            embeddings = normalize(np.array(embeddings).astype('float32'))
            
            dim = embeddings.shape[1]
            index = faiss.IndexHNSWFlat(dim, 32)  # HNSW with M=32
            index.hnsw.efSearch = 50
            index.hnsw.efConstruction = 40
            index.add(embeddings)
            
            save_faiss_index(index, category)
            save_category_data(cat_df, category)
        else:
            continue  # skip empty category

    category_indices[category] = index
    category_id_maps[category] = cat_df

# --- Recommendation function ---
def get_recommendations(input_name, category, top_k=10):
    if category not in category_indices:
        print(f"No index found for category '{category}'")
        return pd.DataFrame()

    input_embedding = model.encode([input_name])
    input_embedding = normalize(np.array(input_embedding).astype('float32'))

    index = category_indices[category]
    distances, indices = index.search(input_embedding, top_k)

    df_cat = category_id_maps[category]
    results = df_cat.iloc[indices[0]][['title', 'imgUrl', 'productURL']].copy()
    results['similarity'] = 1 - distances[0]  # cosine similarity approx
    return results.reset_index(drop=True)

# --- Example usage ---

category_example = categories[0] if categories else None
query_example = "wireless headphones"

if category_example:
    print(f"Example recommendations for query '{query_example}' in category '{category_example}':")
    recs = get_recommendations(query_example, category_example)
    print(recs)
else:
    print("No categories found in dataset.")


Loading index and data for category: 3D Printing  Scanning
Loading index and data for category: Abrasive  Finishing Products
Loading index and data for category: Action Figures, Maquettes  Busts
Loading index and data for category: Action Sports
Loading index and data for category: Air Freshener Supplies
Loading index and data for category: Arts  Crafts Supplies
Loading index and data for category: Artwork
Building index and data for category: Audio/Video Receivers  Amplifiers
Building index and data for category: Automotive
Building index and data for category: Automotive Care
Building index and data for category: Automotive Exterior Accessories
Building index and data for category: Automotive Interior Accessories
Building index and data for category: Automotive Replacement Parts
Building index and data for category: Automotive Tires  Wheels
Building index and data for category: Automotive Tools  Equipment
Building index and data for category: Awards
Building index and data for catego

In [5]:
category_example = categories[0] if categories else None
query_example = "Green Leaf WW3D Wonder Extension Cord Winder, Gray, 150 Foot"

if category_example:
    print(f"Example recommendations for query '{query_example}' in category '{category_example}':")
    recs = get_recommendations(query_example, category_example)
    print(recs)
else:
    print("No categories found in dataset.")

Example recommendations for query 'Green Leaf WW3D Wonder Extension Cord Winder, Gray, 150 Foot' in category '3D Printing  Scanning':
                                               title  \
0  Ruilogod 5Pcs AC220V 10mm Mount Dia Round Indi...   
1  3D Printer Sidewinder X2 and Genius Pro Latest...   
2  SUOFEILAIMU 12mm MGN Miniature Linear Guide Ra...   
3  Wire Rope, Excellent Performance Lightweight a...   
4  Ender-3/CR-10 Series Screen Extension Cable 10...   
5  Professional 1pc MGN9 Linear Guide+1pc MGN9H C...   
6  Hot End PCB Adapter Board 24-pin Cable Kit for...   
7  MOUNAY 24V Brushless Centrifugal Air Blower DC...   
8  Ruilogod DC 12V Power Signal Indicator Pilot L...   
9  Tovmekdjn 24V Brushless Centrifugal Air Blower...   

                                              imgUrl  \
0  https://m.media-amazon.com/images/I/61eMib8hvj...   
1  https://m.media-amazon.com/images/I/613fWr+fWP...   
2  https://m.media-amazon.com/images/I/61vtOjI4D4...   
3  https://m.media-amazon